In [15]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import sys
import time
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import RNN
from keras.layers import LSTM
from keras.utils import np_utils



consumer_key = '0dJqgiNFA3mHYSFymLta1g7Pu'
consumer_secret = 'VZMjNJjlJLhZ0C5KepzyaJk2MFonOfqBX5oYSPu4iez3pqXA91'
access_token = '631788821-Ii9ps4URnZ8b8Hk9gEJ3o84rWTnMKF4k8M9IQX9L'
access_token_secret = 'CJbamHcN04BibPG7Rln6P2wAc6TroPgnNU2EYHBja5DLA'

auth=OAuthHandler(consumer_key,consumer_secret)
auth.set_access_token(access_token,access_token_secret)
api=tweepy.API(auth)

In [ ]:
tweetsdic={}
f = open("demofile.txt", "w")
query='Abraham'

fetchedTweets=api.search(q=query,count=100)
for tweets in fetchedTweets:
    tweetsdic['text']=tweets.text
    f.write(tweets.text.encode('utf-8'))
    time.sleep(10)


text=(open("demofile.txt").read())
text=text.lower()



In [ ]:




# Character embeddings

characters=(sorted(list(set(text))))

char_n={char:n for char , n in enumerate(characters)}
n_char={n:char for char, n in enumerate(characters)}

# Data Preprocessing

x=[]
y=[]

seq_length=100

for i in range(0,len(text)-seq_length,1):
    sequence=text[i:i+seq_length]
    label=text[i+seq_length]
    x.append([char_n[g] for g in sequence])
    y.append(char_n[label])
    
x_mod=np.reshape(x,(len(x),seq_length,1))
x_mod=x_mod/float(len(characters))

y_mod=np_utils.to_categorical(y)

model=Sequential()
model.add(LSTM(40,input_shape=(x_mod.shape[1],y_mod.shape[1]),return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(40,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(40,return_sequences=True))
model.add(Dense(y_mod.shape[1],activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam')

model.fit(x_mod,y_mod,epoch=10,batch_size=50)

model.save_weights('text_generator_40_0.2_40_0.2_40_0.2_10.h5')

model.load_weights('text_generator_40_0.2_40_0.2_40_0.2_10.h5')


# Generating Text


String_mapped=x[99]
full_string=[n_char[j] for j in String_mapped]

for i in range(400):
    x=np.reshape(String_mapped,(1,len(String_mapped),1))
    x=x/float(len(characters))
    
    pred_index=np.argmax(model.predict(x,verbose=0))
    seq=[n_char[f] for f in String_mapped ]
    full_string.append(n_char[pred_index])
    
    String_mapped.append(pred_index)
    String_mapped=String_mapped[1:len(String_mapped)]
    

# txt
txt=" "
for char in full_string:
    txt=txt+char
txt
    
    
    
    
    




